In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('assam_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/guwahati-to-nagaon: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF750F2EEB2+31554]
	(No symbol) [0x00007FF750EA7EE9]
	(No symbol) [0x00007FF750D6872A]
	(No symbol) [0x00007FF750D3D995]
	(No symbol) [0x00007FF750DE44D7]
	(No symbol) [0x00007FF750DFC051]
	(No symbol) [0x00007FF750DDCDD3]
	(No symbol) [0x00007FF750DAA33B]
	(No symbol) [0x00007FF750DAAED1]
	GetHandleVerifier [0x00007FF751238B2D+3217341]
	GetHandleVerifier [0x00007FF751285AF3+3532675]
	GetHandleVerifier [0x00007FF75127B0F0+3489152]
	GetHandleVerifier [0x00007FF750FDE786+750614]
	(No symbol) [0x00007FF750EB376F]
	(No symbol) [0x00007FF750EAEB24]
	(No symbol) [0x00007FF750EAECB2]
	(No symbol) [0x00007FF750E9E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error occurred while

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/biswanath-charali-to-dibrugarh: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF750F2EEB2+31554]
	(No symbol) [0x00007FF750EA7EE9]
	(No symbol) [0x00007FF750D6872A]
	(No symbol) [0x00007FF750D3D995]
	(No symbol) [0x00007FF750DE44D7]
	(No symbol) [0x00007FF750DFC051]
	(No symbol) [0x00007FF750DDCDD3]
	(No symbol) [0x00007FF750DAA33B]
	(No symbol) [0x00007FF750DAAED1]
	GetHandleVerifier [0x00007FF751238B2D+3217341]
	GetHandleVerifier [0x00007FF751285AF3+3532675]
	GetHandleVerifier [0x00007FF75127B0F0+3489152]
	GetHandleVerifier [0x00007FF750FDE786+750614]
	(No symbol) [0x00007FF750EB376F]
	(No symbol) [0x00007FF750EAEB24]
	(No symbol) [0x00007FF750EAECB2]
	(No symbol) [0x00007FF750E9E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]

Error oc

Error occurred while accessing page 5: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.183)
Stacktrace:
	GetHandleVerifier [0x00007FF750F2EEB2+31554]
	(No symbol) [0x00007FF750EA7EE9]
	(No symbol) [0x00007FF750D6872A]
	(No symbol) [0x00007FF750D3D995]
	(No symbol) [0x00007FF750DE44D7]
	(No symbol) [0x00007FF750DFC051]
	(No symbol) [0x00007FF750DDCDD3]
	(No symbol) [0x00007FF750DAA33B]
	(No symbol) [0x00007FF750DAAED1]
	GetHandleVerifier [0x00007FF751238B2D+3217341]
	GetHandleVerifier [0x00007FF751285AF3+3532675]
	GetHandleVerifier [0x00007FF75127B0F0+3489152]
	GetHandleVerifier [0x00007FF750FDE786+750614]
	(No symbol) [0x00007FF750EB376F]
	(No symbol) [0x00007FF750EAEB24]
	(No symbol) [0x00007FF750EAECB2]
	(No symbol) [0x00007FF750E9E17F]
	BaseThreadInitThunk [0x00007FFAD5D0257D+29]
	RtlUserThreadStart [0x00007FFAD7B6AF28+40]



NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 168821,Bharat Benz A/C Seater (2+2),15:45,04h 45m,20:30,3.2,INR 298,9 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,16:00,03h 15m,19:15,4.6,INR 378.92,14 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,3.2,INR 400,10 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Himalayan Travels,,23:40,04h 20m,04:00,1.9,450,
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,,,,,,0,,
5,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,WARISPIYA TRAVELS,Bharat Benz A/C Seater (2+2),16:15,03h 25m,19:40,4.2,INR 360,15 Seats available
6,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),18:00,05h 30m,23:30,3.3,400,22 Seats available
7,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,3.6,400,18 Seats available
8,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),,20:15,05h 00m,01:15,3.4,INR 450,
9,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,,,,,,,,
